# Data Collection

This assumes that you have registered an account with Kaggle, and have created a json key to interact with Kaggle's API.  

In [8]:
import os
import shutil
import pandas as pd
import numpy as np

In [9]:
os.chdir(os.path.expanduser('~'))
os.chdir('Desktop/skin')

current_dir = os.getcwd()
current_dir

'/home/pxp/Desktop/skin'

In [10]:
meta_data = pd.read_csv("inputs/skin cancer/HAM10000_metadata.csv")
meta_data.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear


# Data Preparation

## Split train validation test set

In [11]:
image_dir_path = 'inputs/skin cancer/Skin Cancer/Skin Cancer' 
train_dir = 'inputs/work/train'
test_dir = 'inputs/work/test'
val_dir = 'inputs/work/validate'

os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)

cancer_types = meta_data["dx"].unique()
for cancer_type in cancer_types:
    os.makedirs(os.path.join(train_dir, cancer_type), exist_ok=True)
    os.makedirs(os.path.join(test_dir, cancer_type), exist_ok=True)
    os.makedirs(os.path.join(val_dir, cancer_type), exist_ok=True)

In [12]:
for dpath in [train_dir, test_dir, val_dir]:
    for root, dirs, files in os.walk(dpath):
        for file in files:
            os.remove(os.path.join(root, file))

In [13]:
meta_data = meta_data.sample(frac=1).reset_index(drop=True)

train_count = int(meta_data.shape[0]*.7)
test_count = int(meta_data.shape[0]*.2)

counts = {}

for i in range(meta_data.shape[0]):
    image_path = os.path.join(image_dir_path, meta_data["image_id"][i]+".jpg")
    if i < train_count:
        target = os.path.join(train_dir, meta_data["dx"][i])
    elif i < train_count + test_count:
        target = os.path.join(test_dir, meta_data["dx"][i])
    else:
        target = os.path.join(val_dir, meta_data["dx"][i])
        
    if not target in counts:
        counts[target] = 0
    else:
        counts[target] += 1
    if counts[target] > 200:
        continue
    shutil.copy(image_path, target)

In [14]:
paths = [train_dir, test_dir, val_dir]
names = ['TRAIN', 'TEST', 'VALIDATE']
nums = pd.DataFrame(columns=names, index=cancer_types)

for name in names: 
    for cancer_type in cancer_types:
        nums.loc[cancer_type, name] = len(os.listdir(os.path.join(paths[names.index(name)], cancer_type)))

nums.loc['TOTAL'] = nums.sum()
nums['TOTAL'] = nums.sum(axis=1)

print('Files present:')
nums 

Files found:


,TRAIN,TEST,VALIDATE,TOTAL
bkl,790,198,111,1099
nv,1001,1001,681,2683
df,79,22,14,115
mel,774,230,109,1113
vasc,97,29,16,142
bcc,366,110,38,514
akiec,230,64,33,327
TOTAL,3337,1654,1002,5993
